1. Cases per Hospital
2. Types of Hospital
3. How many hospitals fall in each cities
4. How many cases did each hospitals conduct over cities
5. How often did the patient visit hospital?
6. Do the patient prefer same hospital everytime?
7. Which is the most visited department
8. Spread of Admissions
9. Do older people stay longer ?
10. Which age people pays more?
11. Is initial admission deposit related to their stay?
12. How much is deposit related to age and stay
13. How long patient stay based on severity
14. Are Age & Stay Days related to Severeness
15. Is bed alloted based on severeness
16. Are there more visitors for patients with severe illness
17. Do hospitals provide extra rooms for long stay
18. How many hospital types fall in each region

In [5]:
import numpy as np 
import pandas as pd
import altair as alt
from vega_datasets import data

In [12]:
hosp=pd.read_csv('train_data.csv')
hosp.sample(5)

,case_id,Hospital_code,Hospital_type_code,City_Code_Hospital,Hospital_region_code,Available Extra Rooms in Hospital,Department,Ward_Type,Ward_Facility_Code,Bed Grade,patientid,City_Code_Patient,Type of Admission,Severity of Illness,Visitors with Patient,Age,Admission_Deposit,Stay
138559,138560,16,c,3,Z,4,gynecology,R,A,2.0,93481,9.0,Emergency,Moderate,2,31-40,3581.0,0-10
70094,70095,10,e,1,X,2,anesthesia,S,E,3.0,58245,15.0,Trauma,Moderate,4,41-50,5087.0,31-40
238541,238542,25,e,1,X,4,gynecology,Q,E,4.0,19899,10.0,Trauma,Minor,4,41-50,5181.0,31-40
42648,42649,23,a,6,X,2,radiotherapy,Q,F,4.0,1693,1.0,Trauma,Moderate,3,51-60,4306.0,11-20
193165,193166,11,b,2,Y,2,gynecology,Q,D,4.0,34268,8.0,Trauma,Minor,2,61-70,4754.0,11-20


In [19]:
print('Number of unique hospitals: ',hosp['Hospital_code'].nunique())
print('Number of cases present in the dataset: ',len(hosp))
print('Number of unique patients present in the dataset: ',hosp['patientid'].nunique())

Number of unique hospitals:  32
Number of cases present in the dataset:  318438
Number of unique patients present in the dataset:  92017


In [27]:
hosp_visits=hosp['Hospital_code'].value_counts().to_frame().reset_index().rename(columns={'index':'Hospital_code','Hospital_code':'Total'})
hosp_visits.head()

,Hospital_code,Total
0,26,33076
1,23,26566
2,19,21219
3,6,20425
4,11,17328
